# EEG Feature Selection

In [10]:
# EEG Feature Selection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mne
from scipy import signal
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from mne.decoding import CSP
import json
import os
# Set plotting style
plt.style.use('ggplot')
sns.set(font_scale=1.2)
sns.set_style('whitegrid')

# Set random seed for reproducibility
np.random.seed(42)

## 1. Load Dataset
First, let's load the EEG data and split information:

In [21]:
import os
import json
import numpy as np
import pandas as pd
import mne
from mne.io import read_raw_eeglab

# Load the k-fold splits file
with open('kfold_splits.json', 'r') as f:
    kfold_data = json.load(f)

# Get the first fold
first_fold = kfold_data["fold_1"]
train_subjects = first_fold["train"]
validation_subjects = first_fold["validation"]

print(f"Training subjects: {len(train_subjects)}")
print(f"Validation subjects: {len(validation_subjects)}")
print(f"Group distribution in training: {first_fold['train_group_counts']}")

# Load participants.tsv to get group information
participants_df = pd.read_csv('dataset/participants.tsv', sep='\t')
print(f"Loaded participant information for {len(participants_df)} subjects")

# Function to load EEG data
def load_data(subject_ids, data_path='dataset', use_derivatives=True):
    """
    Load EEG data for given subjects
    
    Parameters:
    -----------
    subject_ids : list
        List of subject IDs to load
    data_path : str
        Path to the data directory
    use_derivatives : bool
        Whether to use the preprocessed data from the derivatives folder
        
    Returns:
    --------
    X : numpy array
        EEG data
    y : numpy array
        Labels (0 for Alzheimer's, 1 for FTD, 2 for Control, based on group A/F/C)
    sfreq : float
        Sampling frequency
    """
    X = []
    y = []
    subject_info = []  # To keep track of which subjects were loaded
    
    # Dictionary to map group codes to numeric labels
    group_to_label = {'A': 0, 'F': 1, 'C': 2}
    
    for subject_id in subject_ids:
        # Get group from participants.tsv
        if subject_id in participants_df['participant_id'].values:
            try:
                group = participants_df.loc[participants_df['participant_id'] == subject_id, 'Group'].values[0]
            except KeyError:
                # Try alternative column names
                for col in ['group', 'GROUP', 'diagnosis', 'DIAGNOSIS']:
                    if col in participants_df.columns:
                        group = participants_df.loc[participants_df['participant_id'] == subject_id, col].values[0]
                        break
                else:
                    # If no group column found, use the fold data
                    group = None
        else:
            group = None
            
        # If group not found in participants.tsv, try to infer from fold data
        if group is None:
            # Try to infer from train_group_counts
            for g in ['A', 'F', 'C']:
                if g in first_fold['train_group_counts']:
                    # This is a simplification - we're just assigning based on the first available group
                    group = g
                    break
            
            if group is None:
                print(f"Warning: Could not determine group for {subject_id}, using default 'A'")
                group = 'A'  # Default to Alzheimer's
        
        # Determine file path based on whether to use derivatives or not
        if use_derivatives:
            file_path = os.path.join(data_path, 'derivatives', subject_id, 'eeg', f"{subject_id}_task-eyesclosed_eeg.set")
        else:
            file_path = os.path.join(data_path, subject_id, 'eeg', f"{subject_id}_task-eyesclosed_eeg.set")
        
        # Check if file exists
        if not os.path.exists(file_path):
            # Try the alternative location
            if use_derivatives:
                alt_path = os.path.join(data_path, subject_id, 'eeg', f"{subject_id}_task-eyesclosed_eeg.set")
            else:
                alt_path = os.path.join(data_path, 'derivatives', subject_id, 'eeg', f"{subject_id}_task-eyesclosed_eeg.set")
                
            if os.path.exists(alt_path):
                file_path = alt_path
            else:
                print(f"Could not find data for {subject_id}, skipping")
                continue
        
        try:
            # Load the .set file using MNE
            raw = mne.io.read_raw_eeglab(file_path, preload=True)
            X.append(raw.get_data())
            
            # Convert group to numeric label
            label = group_to_label.get(group, 0)  # Default to 0 if group not found
            y.append(label)
            subject_info.append(subject_id)
            
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
    
    if not X:
        raise ValueError("No data was loaded successfully")
    
    # Get sampling frequency from the last loaded file
    sfreq = raw.info['sfreq']
    
    print(f"Successfully loaded {len(X)} subjects: {subject_info}")
    
    return np.array(X), np.array(y), sfreq

# Load training data
print("\nLoading training data...")
X_train, y_train, sfreq = load_data(train_subjects, use_derivatives=True)
print(f"Loaded {len(X_train)} training samples")
print(f"Sampling frequency: {sfreq} Hz")

# Load validation data
print("\nLoading validation data...")
X_val, y_val, _ = load_data(validation_subjects, use_derivatives=True)
print(f"Loaded {len(X_val)} validation samples")

# Print class distribution
print("\nClass distribution:")
for label, count in zip(*np.unique(y_train, return_counts=True)):
    print(f"  Training - Class {label}: {count} samples")
for label, count in zip(*np.unique(y_val, return_counts=True)):
    print(f"  Validation - Class {label}: {count} samples")

# Print data shape
print(f"\nData shape:")
print(f"  X_train shape: {X_train.shape}")
print(f"  X_val shape: {X_val.shape}")

Training subjects: 70
Validation subjects: 18
Group distribution in training: {'A': 28, 'F': 18, 'C': 24}
Loaded participant information for 88 subjects

Loading training data...


<ipython-input-21-a0775936d69c>:107: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file_path, preload=True)
<ipython-input-21-a0775936d69c>:107: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file_path, preload=True)
<ipython-input-21-a0775936d69c>:107: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file_path, preload=True)
<ipython-input-21-a0775936d69c>:107: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file_path, preload=True)
<ipython-input-21-a0775936d69c>:107: RuntimeWarn

Successfully loaded 70 subjects: ['sub-021', 'sub-026', 'sub-019', 'sub-010', 'sub-017', 'sub-028', 'sub-016', 'sub-029', 'sub-011', 'sub-027', 'sub-018', 'sub-020', 'sub-033', 'sub-034', 'sub-035', 'sub-004', 'sub-025', 'sub-014', 'sub-012', 'sub-015', 'sub-023', 'sub-024', 'sub-006', 'sub-001', 'sub-008', 'sub-031', 'sub-036', 'sub-007', 'sub-043', 'sub-044', 'sub-042', 'sub-060', 'sub-058', 'sub-051', 'sub-056', 'sub-057', 'sub-050', 'sub-059', 'sub-061', 'sub-040', 'sub-049', 'sub-048', 'sub-046', 'sub-039', 'sub-037', 'sub-064', 'sub-063', 'sub-038', 'sub-053', 'sub-054', 'sub-062', 'sub-065', 'sub-086', 'sub-072', 'sub-075', 'sub-081', 'sub-080', 'sub-074', 'sub-087', 'sub-067', 'sub-069', 'sub-068', 'sub-066', 'sub-078', 'sub-082', 'sub-085', 'sub-071', 'sub-084', 'sub-083', 'sub-079']


<ipython-input-21-a0775936d69c>:107: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file_path, preload=True)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (70, 19) + inhomogeneous part.

## 2. Filtering and Preprocessing

In [13]:
def preprocess_eeg(X, sfreq, l_freq=1.0, h_freq=45.0):
    """
    Basic preprocessing for EEG data:
    - Bandpass filtering
    - Notch filtering (for line noise)
    
    Parameters:
    X : array of EEG data with shape (n_subjects, n_channels, n_samples)
    sfreq : sampling frequency
    l_freq, h_freq : low and high frequency cutoffs for bandpass filter
    
    Returns:
    X_filtered : filtered EEG data
    """
    X_filtered = []
    
    for eeg_data in X:
        # Create a raw object for easier processing
        ch_names = [f'ch{i}' for i in range(eeg_data.shape[0])]
        info = mne.create_info(ch_names=ch_names, sfreq=sfreq)
        raw = mne.io.RawArray(eeg_data, info)
        
        # Apply bandpass filter
        raw.filter(l_freq=l_freq, h_freq=h_freq, method='fir')
        
        # Apply notch filter (for 60 Hz line noise, adjust if needed)
        raw.notch_filter(freqs=60, method='fir')
        
        # Get filtered data
        X_filtered.append(raw.get_data())
    
    return np.array(X_filtered)

# Apply filtering
X_train_filtered = preprocess_eeg(X_train, sfreq)
print(f"Filtered data shape: {X_train_filtered.shape}")

NameError: name 'X_train' is not defined

## 3. Frequency Band Power Features

In [ ]:
def extract_band_powers(X, sfreq):
    """
    Extract power in standard EEG frequency bands
    
    Parameters:
    X : array of EEG data with shape (n_subjects, n_channels, n_samples)
    sfreq : sampling frequency
    
    Returns:
    band_powers : array of band power features
    feature_names : list of feature names
    """
    # Define frequency bands
    bands = {
        'delta': (0.5, 4),
        'theta': (4, 8),
        'alpha': (8, 13),
        'beta': (13, 30),
        'gamma': (30, 45)
    }
    
    all_band_powers = []
    feature_names = []
    
    for subject_idx, eeg_data in enumerate(X):
        subject_powers = []
        
        # For each channel
        for ch_idx in range(eeg_data.shape[0]):
            channel_data = eeg_data[ch_idx]
            
            # Calculate PSD using Welch's method
            freqs, psd = signal.welch(channel_data, sfreq, nperseg=int(sfreq * 2))
            
            # Extract band powers
            ch_powers = []
            for band_name, (fmin, fmax) in bands.items():
                # Find frequency indices within band
                idx_band = np.logical_and(freqs >= fmin, freqs <= fmax)
                # Calculate average power in band
                band_power = np.mean(psd[idx_band])
                ch_powers.append(band_power)
                
                # Create feature names (only once)
                if subject_idx == 0 and ch_idx == 0:
                    feature_names.append(f"ch{ch_idx}_{band_name}")
            
            subject_powers.extend(ch_powers)
        
        all_band_powers.append(subject_powers)
    
    return np.array(all_band_powers), feature_names

# Extract band powers
band_powers, power_feature_names = extract_band_powers(X_train_filtered, sfreq)
print(f"Band power features shape: {band_powers.shape}")
print(f"Example features: {power_feature_names[:10]}")

## 4. Power Ratios

In [ ]:
def extract_power_ratios(X, sfreq):
    """
    Calculate ratios between different frequency bands
    
    Parameters:
    X : array of EEG data with shape (n_subjects, n_channels, n_samples)
    sfreq : sampling frequency
    
    Returns:
    ratio_features : array of power ratio features
    ratio_names : list of feature names
    """
    # Define frequency bands
    bands = {
        'delta': (0.5, 4),
        'theta': (4, 8),
        'alpha': (8, 13),
        'beta': (13, 30),
        'gamma': (30, 45)
    }
    
    # Define ratios to calculate
    ratios = [
        ('theta', 'beta'),     # theta/beta ratio
        ('alpha', 'theta'),    # alpha/theta ratio
        ('delta', 'theta'),    # delta/theta ratio
        ('alpha', 'beta'),     # alpha/beta ratio
        ('delta', 'alpha'),    # delta/alpha ratio
    ]
    
    all_ratios = []
    ratio_names = []
    
    for subject_idx, eeg_data in enumerate(X):
        subject_ratios = []
        
        # For each channel
        for ch_idx in range(eeg_data.shape[0]):
            channel_data = eeg_data[ch_idx]
            
            # Calculate PSD
            freqs, psd = signal.welch(channel_data, sfreq, nperseg=int(sfreq * 2))
            
            # Calculate band powers
            band_powers = {}
            for band_name, (fmin, fmax) in bands.items():
                idx_band = np.logical_and(freqs >= fmin, freqs <= fmax)
                band_powers[band_name] = np.mean(psd[idx_band])
            
            # Calculate ratios
            ch_ratios = []
            for num_band, denom_band in ratios:
                # Avoid division by zero
                if band_powers[denom_band] > 0:
                    ratio = band_powers[num_band] / band_powers[denom_band]
                else:
                    ratio = 0
                ch_ratios.append(ratio)
                
                # Create feature names (only once)
                if subject_idx == 0 and ch_idx == 0:
                    ratio_names.append(f"ch{ch_idx}_{num_band}_{denom_band}_ratio")
            
            subject_ratios.extend(ch_ratios)
        
        all_ratios.append(subject_ratios)
    
    return np.array(all_ratios), ratio_names

# Extract power ratios
power_ratios, ratio_feature_names = extract_power_ratios(X_train_filtered, sfreq)
print(f"Power ratio features shape: {power_ratios.shape}")
print(f"Example ratio features: {ratio_feature_names[:10]}")

## 5. Common Spatial Patterns (CSP)

In [ ]:
def extract_csp_features(X, y, n_components=4):
    """
    Extract features using Common Spatial Patterns
    
    Parameters:
    X : array of EEG data with shape (n_subjects, n_channels, n_samples)
    y : array of labels
    n_components : number of CSP components to extract
    
    Returns:
    csp_features : array of CSP features
    csp : fitted CSP object
    """
    # Initialize CSP
    csp = CSP(n_components=n_components, reg=None, log=True, norm_trace=False)
    
    # Fit and transform
    csp_features = csp.fit_transform(X, y)
    
    # Create feature names
    feature_names = [f"CSP_{i}" for i in range(n_components)]
    
    return csp_features, feature_names, csp

# Extract CSP features
csp_features, csp_feature_names, csp = extract_csp_features(X_train_filtered, y_train)
print(f"CSP features shape: {csp_features.shape}")
print(f"CSP feature names: {csp_feature_names}")

## 6. Combine Features and Evaluate

In [ ]:
# Combine all features
combined_features = np.hstack((band_powers, power_ratios, csp_features))
all_feature_names = power_feature_names + ratio_feature_names + csp_feature_names
print(f"Combined features shape: {combined_features.shape}")

# Select best features using ANOVA F-test
selector = SelectKBest(f_classif, k=20)
X_selected = selector.fit_transform(combined_features, y_train)
selected_indices = selector.get_support(indices=True)
selected_features = [all_feature_names[i] for i in selected_indices]

print("Top 20 features selected by ANOVA F-test:")
for i, feature in enumerate(selected_features):
    print(f"{i+1}. {feature}")

# Evaluate feature importance with Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(combined_features, y_train)

# Get feature importances
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
top_indices = indices[:20]
top_features = [all_feature_names[i] for i in top_indices]

print("\nTop 20 features by Random Forest importance:")
for i, (feature, importance) in enumerate(zip(top_features, importances[top_indices])):
    print(f"{i+1}. {feature}: {importance:.4f}")

# Visualize feature importance
plt.figure(figsize=(12, 8))
plt.barh(range(20), importances[top_indices], align='center')
plt.yticks(range(20), top_features)
plt.xlabel('Feature Importance')
plt.title('Top 20 Features by Random Forest Importance')
plt.tight_layout()
plt.show()

# Evaluate the selected features
def evaluate_features(X, y, n_folds=5):
    """Evaluate features using cross-validation"""
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    scores = cross_val_score(clf, X, y, cv=n_folds, scoring='accuracy')
    return scores

# Evaluate with all combined features
all_scores = evaluate_features(combined_features, y_train)
print(f"\nAll features ({combined_features.shape[1]}): {np.mean(all_scores):.4f} ± {np.std(all_scores):.4f}")

# Evaluate with selected features
selected_scores = evaluate_features(X_selected, y_train)
print(f"Selected features (20): {np.mean(selected_scores):.4f} ± {np.std(selected_scores):.4f}")

# Evaluate each feature type separately
band_scores = evaluate_features(band_powers, y_train)
print(f"Band power features ({band_powers.shape[1]}): {np.mean(band_scores):.4f} ± {np.std(band_scores):.4f}")

ratio_scores = evaluate_features(power_ratios, y_train)
print(f"Power ratio features ({power_ratios.shape[1]}): {np.mean(ratio_scores):.4f} ± {np.std(ratio_scores):.4f}")

csp_scores = evaluate_features(csp_features, y_train)
print(f"CSP features ({csp_features.shape[1]}): {np.mean(csp_scores):.4f} ± {np.std(csp_scores):.4f}")

## 7. Feature Selection Summary

In [ ]:
# Compare different feature types
feature_types = ['Band Powers', 'Power Ratios', 'CSP', 'Selected 20', 'All Combined']
accuracies = [
    np.mean(band_scores),
    np.mean(ratio_scores),
    np.mean(csp_scores),
    np.mean(selected_scores),
    np.mean(all_scores)
]
std_devs = [
    np.std(band_scores),
    np.std(ratio_scores),
    np.std(csp_scores),
    np.std(selected_scores),
    np.std(all_scores)
]

plt.figure(figsize=(10, 6))
bars = plt.bar(feature_types, accuracies, yerr=std_devs, capsize=10)
plt.ylabel('Cross-Validation Accuracy')
plt.title('Comparison of Feature Types')
plt.ylim(0.5, 1.0)  # Adjust as needed

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
            f'{height:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Summary table
results_df = pd.DataFrame({
    'Feature Type': feature_types,
    'Number of Features': [band_powers.shape[1], power_ratios.shape[1], 
                          csp_features.shape[1], 20, combined_features.shape[1]],
    'Mean Accuracy': accuracies,
    'Std Dev': std_devs
})

print("Feature Selection Summary:")
print(results_df)

# Save the selected features list to a file for future use
with open('selected_features.txt', 'w') as f:
    f.write("Selected features by ANOVA F-test:\n")
    for feature in selected_features:
        f.write(f"{feature}\n")
    
    f.write("\nSelected features by Random Forest importance:\n")
    for feature in top_features:
        f.write(f"{feature}\n")

print("\nSelected features saved to 'selected_features.txt'")